## PASO 0 EXTRACCION DE DATOS (df_v0)

In [3]:
import pandas as pd
import numpy as np
from datetime import datetime as dt
import re
import mysql.connector

## PASO 4 CREACIÓN DE BBDD EN MYSQL

    create database factoria_jamones;
    use factoria_jamones;

    CREATE TABLE `raza` (
      `id_raza` integer auto_increment,
      `desc_raza` varchar(100),
      PRIMARY KEY (`id_raza`)
    );

    CREATE TABLE `matadero` (
      `id_matadero` integer,
      PRIMARY KEY (`id_matadero`)
    );

    CREATE TABLE `alimentacion` (
      `id_alimentacion` integer auto_increment,
      `desc_alimentacion` varchar(100),
      PRIMARY KEY (`id_alimentacion`)
    );

    CREATE TABLE `partida` (
      `id_partida` integer,
      `id_raza` integer,
      `añada` integer,
      `acido_oleico` float,
      `acido_palmitico` float,
      `acido_estearico` float,
      `acido_linoleico` float,
      `peso_varios` float,
      `peso_vientres` float,
      `peso_cerdos` float,
      `no_animales` integer,
      `nacimiento` date,
      `meses_sacrificio` integer,
      `id_matadero` integer,
      `id_alimentacion` integer,
      `fecha_entrada` date,
      PRIMARY KEY (`id_partida`),
      FOREIGN KEY (`id_raza`) REFERENCES `Raza`(`id_raza`),
      FOREIGN KEY (`id_matadero`) REFERENCES `Matadero`(`id_matadero`),
      FOREIGN KEY (`id_alimentacion`) REFERENCES `Alimentacion`(`id_alimentacion`)
    );

    CREATE TABLE `articulo_Final` (
      `id_articulo` integer auto_increment,
      `desc_articulo` varchar(100),
      PRIMARY KEY (`id_articulo`)
    );

    CREATE TABLE `tipo` (
      `id_tipo` integer auto_increment,
      `desc_tipo` varchar(100),
      PRIMARY KEY (`id_tipo`)
    );

    CREATE TABLE `pieza` (
      `id_pieza` integer auto_increment,
      `id_tipo` integer,
      `id_partida` integer,
      `peso_salida` float,
      `fecha_salida` date,
      `id_articulo` integer,
      `numero_serie_lc` integer,
      `tipo_pesada_salida` varchar(100),
      `estado_producto_salida` varchar(100),
      PRIMARY KEY (`id_pieza`),
      FOREIGN KEY (`id_articulo`) REFERENCES `Articulo_Final`(`id_articulo`),
      FOREIGN KEY (`id_partida`) REFERENCES `Partida`(`id_partida`),
      FOREIGN KEY (`id_tipo`) REFERENCES `Tipo`(`id_tipo`)
    );


## PASO 5 INTRODUCCIÓN DE DATOS A LA BBDD

 Las tablas las llenaremos en el mismo orden que se crearon en la base de la siguiente manera:

### .- Paso 5.1 Tablas de categoría se introduciran mediante una lista creada aplicada a la columna que corresponda con el método .unique():
        raza
        matadero
        alimentacion
        tipo_pesada_salida
        tipo_pesada_entrada
        estado_salida
        tipo_pieza

### .- Paso 5.2 Tabla partida: previo checkeo de valores únicos, hay que crear la tabla en python mediante una agrupación groupby.

        
### .- Paso 5.3 Tabla pieza: es la tabla con la que estamos trabajando unida a la tabla partida y todas las categorías.

    create database factoria_jamones;
    use factoria_jamones;

    CREATE TABLE `raza` (
      `id_raza` integer,
      `desc_raza` varchar(100),
      PRIMARY KEY (`id_raza`)
    );

    CREATE TABLE `matadero` (
      `id_matadero` integer,
      `desc_matadero` varchar(100),
      PRIMARY KEY (`id_matadero`)
    );

    CREATE TABLE `alimentacion` (
      `id_alimentacion` integer,
      `desc_alimentacion` varchar(100),
      PRIMARY KEY (`id_alimentacion`)
    );

    CREATE TABLE `tipo_pesada_entrada` (
      `id_tipo_pesada_entrada` integer,
      `desc_tipo_pesada_entrada` varchar(100),
      PRIMARY KEY (`id_tipo_pesada_entrada`)
    );

    CREATE TABLE `partida` (
      `id_partida` integer,
      `id_raza` integer,
      `añada` integer,
      `acido_oleico` float,
      `acido_palmitico` float,
      `acido_estearico` float,
      `acido_linoleico` float,
      `peso_varios` float,
      `peso_vientres` float,
      `peso_cerdos` float,
      `peso_neto_canal` float,
      `no_animales` integer,
      `nacimiento` date,
      `id_matadero` integer,
      `id_alimentacion` integer,
      `id_tipo_pesada_entrada` integer,
      `fecha_entrada` date,
      PRIMARY KEY (`id_partida`),
      FOREIGN KEY (`id_raza`) REFERENCES `raza`(`id_raza`),
      FOREIGN KEY (`id_matadero`) REFERENCES `matadero`(`id_matadero`),
      FOREIGN KEY (`id_tipo_pesada_entrada`) REFERENCES `tipo_pesada_entrada`(`id_tipo_pesada_entrada`),
      FOREIGN KEY (`id_alimentacion`) REFERENCES `alimentacion`(`id_alimentacion`)
    );



    CREATE TABLE `tipo_pesada_salida` (
      `id_tipo_pesada_salida` integer,
      `desc_tipo_pesada_salida` varchar(100),
      PRIMARY KEY (`id_tipo_pesada_salida`)
    );

    CREATE TABLE `estado_salida` (
      `id_estado_salida` integer,
      `desc_estado_salida` varchar(100),
      PRIMARY KEY (`id_estado_salida`)
    );

    CREATE TABLE `tipo_pieza` (
      `id_tipo_pieza` integer,
      `desc_tipo_pieza` varchar(100),
      PRIMARY KEY (`id_tipo_pieza`)
    );

    CREATE TABLE `pieza` (
      `id_pieza` integer auto_increment,
      `no_serie` varchar(100),
      `id_tipo_pieza` integer,
      `id_partida` integer,
      `peso_salida` float, 
      `fecha_salida` date,
      `id_tipo_pesada_salida` integer,
      `id_estado_salida` integer,
      `numero_serie_lc` integer,
      PRIMARY KEY (`id_pieza`),
      FOREIGN KEY (`id_tipo_pesada_salida`) REFERENCES `tipo_pesada_salida`(`id_tipo_pesada_salida`),
      FOREIGN KEY (`id_estado_salida`) REFERENCES `estado_salida`(`id_estado_salida`),
      FOREIGN KEY (`id_partida`) REFERENCES `partida`(`id_partida`),
      FOREIGN KEY (`id_tipo_pieza`) REFERENCES `tipo_pieza`(`id_tipo_pieza`)
    );

## .- Paso 5.1 Tablas de categoría se introduciran mediante una diccionario creado aplicada a la columna que corresponda con el método unique():

    raza
    matadero
    alimentacion
    tipo_pesada_salida
    tipo_pesada_entrada
    estado_salida
    tipo_pieza

In [27]:
def DiccCats(listadoCategorias):

    categorias = {}
    n = 0
    for values in listadoCategorias:
        categorias[values] = n
        n+=1
    return categorias

In [ ]:



def IntroTablasCategorias(diccionario, nombreColumna):
    import mysql.connector
    
    sqlcategoria = "INSERT INTO factoria_jamones."+str(nombreColumna)+"(id_"+str(nombreColumna)+", desc_"+str(nombreColumna)+") VALUES (%s, %s)"
    conex = mysql.connector.connect(
            host = "localhost",
            user = "root", 
            password = "",
            database ="factoria_jamones",
            port = 3306,
            autocommit=True)
    transaccion = conex.cursor()

    count = 0
    for values in diccionario:
        try:
            transaccion.execute(sqlcategoria, (diccionario[values], values))
        except: 
            print("elemento número "+ str(count+1) +" ya existe en base de datos")
            count+=1
    conex.close()

In [23]:
df_InDt1 = df_DtTyp.copy()
df_InDt1

,TipoArticuloEscandallo,Añada,Partida,Raza,Alimentacion,NumeroSerieLc,AcidoOleico,AcidoPalmitico,AcidoEstearico,AcidoLinoleico,...,CodigoMatadero,NroAnimales,PesoEntradaEntrada,TipoPesada,PesoSalida,DescripcionArticulo,FechaEntrada,FechaSalida,TipoPesadaSalida,EstadoProductoSalida
480,Jamón,2014,1025,50% IBERICO,BELLOTA,140121025013,57.4,18.99,8.23,8.85,...,1,130,NaN,Grupal,8.755,JAMON DE BELLOTA IBERICO PASSION,2014-03-20,2018-06-04,Pesada PIEZA ENTERA,Salida Correcta
481,Jamón,2014,1025,50% IBERICO,BELLOTA,140121025117,57.4,18.99,8.23,8.85,...,1,130,NaN,Grupal,7.755,"JAMON BELLOTA IBERICO 75% RAZA IBERICA75% 7,0-...",2014-03-20,2018-10-31,Pesada PIEZA ENTERA,Salida Correcta
482,Jamón,2014,1025,50% IBERICO,BELLOTA,140121025205,57.4,18.99,8.23,8.85,...,1,130,NaN,Grupal,7.755,"JAMON BELLOTA IBERICO 75% RAZA IBERICA75% 7,0-...",2014-03-20,2018-10-31,Pesada PIEZA ENTERA,Salida Correcta
483,Jamón,2014,1025,50% IBERICO,BELLOTA,140121025235,57.4,18.99,8.23,8.85,...,1,130,NaN,Grupal,7.400,JAMON IBERICO BELLOTA PIEZA UN DONIBERIIBERICO...,2014-03-20,2017-12-01,Pesada PIEZA ENTERA,Salida Correcta
484,Paleta,2014,1025,50% IBERICO,BELLOTA,141121025162,57.4,18.99,8.23,8.85,...,1,130,NaN,Grupal,6.640,PALETA DE BELLOTA IBÉRICA PASSION,2014-03-20,2017-08-28,Pesada PIEZA ENTERA,Salida Correcta
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158723,Paleta,2018,999,75% IBERICO,BELLOTA,237030576292,NaN,NaN,NaN,NaN,...,1,65,NaN,Grupal,6.383,PALETA DE BELLOTA IBÉRICA,2018-02-12,2021-04-13,Pesada PIEZA ENTERA,Salida Correcta
158724,Paleta,2018,999,75% IBERICO,BELLOTA,237030576292,NaN,NaN,NaN,NaN,...,1,65,NaN,Grupal,3.160,PALETA DE BELLOTA IBÉRICA,2018-02-12,2021-04-13,Pesada PIEZA ENTERA,Salida Correcta
158725,Paleta,2018,999,75% IBERICO,BELLOTA,237030576292,NaN,NaN,NaN,NaN,...,1,65,NaN,Grupal,6.383,PALETA DE BELLOTA IBÉRICA DESH.,2018-02-12,2021-04-16,Pesada pieza deshuesada,Salida Correcta
158726,Paleta,2018,999,75% IBERICO,BELLOTA,237030576292,NaN,NaN,NaN,NaN,...,1,65,NaN,Grupal,3.160,PALETA DE BELLOTA IBÉRICA DESH.,2018-02-12,2021-04-16,Pesada pieza deshuesada,Salida Correcta


In [24]:
df_InDt2 = df_InDt1.copy()

# Checkeamos valores de categorías
print(df_InDt2["Raza"].unique())
print(df_InDt2["CodigoMatadero"].unique())
print(df_InDt2["Alimentacion"].unique())
print(df_InDt2["TipoPesadaSalida"].unique())
print(df_InDt2["EstadoProductoSalida"].unique())
print(df_InDt2["TipoPesada"].unique())
print(df_InDt2["TipoArticuloEscandallo"].unique())

['50% IBERICO' '75% IBERICO' '100% IBERICO']
[ 1 -1  3  2]
['BELLOTA' 'CEBO DE CAMPO' 'CEBO']
['Pesada PIEZA ENTERA' 'Pesada pieza deshuesada'
 'Pesada blisters para caja']
['Salida Correcta' 'Salida Prematura']
['Grupal' 'Individual']
['Jamón' 'Paleta']


In [25]:
# Introducimos valores

IntroTablasCategorias(DiccCats(df_InDt2["Raza"].unique()),"raza")
IntroTablasCategorias(DiccCats(df_InDt2["CodigoMatadero"].unique().astype("object")),"matadero")
IntroTablasCategorias(DiccCats(df_InDt2["Alimentacion"].unique()),"alimentacion")
IntroTablasCategorias(DiccCats(df_InDt2["TipoPesadaSalida"].unique()),"tipo_pesada_salida")
IntroTablasCategorias(DiccCats(df_InDt2["TipoPesada"].unique()),"tipo_pesada_entrada")
IntroTablasCategorias(DiccCats(df_InDt2["EstadoProductoSalida"].unique()),"estado_salida")
IntroTablasCategorias(DiccCats(df_InDt2["TipoArticuloEscandallo"].unique()),"tipo_pieza")

NameError: name 'IntroTablasCategorias' is not defined

In [28]:
# Para introducir correctamente los datos en las tablas sustituimos 
# los valores de las categorías por sus ids en el dataframe principal.

df_InDt3 = df_InDt2.copy()
lista = ["Raza","CodigoMatadero","Alimentacion","TipoPesadaSalida","TipoPesada","EstadoProductoSalida","TipoArticuloEscandallo"]

for columns in lista:
    df_InDt3[columns].replace(to_replace=DiccCats(df_InDt3[columns].unique()), inplace=True)
    

print(df_InDt3["Raza"].unique())
print(df_InDt3["CodigoMatadero"].unique())
print(df_InDt3["Alimentacion"].unique())
print(df_InDt3["TipoPesadaSalida"].unique())
print(df_InDt3["EstadoProductoSalida"].unique())
print(df_InDt3["TipoPesada"].unique())
print(df_InDt3["TipoArticuloEscandallo"].unique())


[0 1 2]
[0 1 2 3]
[0 1 2]
[0 1 2]
[0 1]
[0 1]
[0 1]


## .- Paso 5.2 Tabla partida: previo checkeo de valores únicos, hay que crear la tabla en python mediante una agrupación groupby.

In [29]:
# Hay que comprobar que los siguientes valores sean únicos para cada partida

# TipoArticuloEscandallo      2
# Añada                       1 [1]
# Partida                     1 
# Raza                        1 [3]
# Alimentacion                1 [4]
# NumeroSerieLc             156
# AcidoOleico                 0 [6]
# AcidoPalmitico              0 [7]
# AcidoEstearico              0 [8]
# AcidoLinoleico              0 [9]
# PesoVarios                  1 [10]
# Pesovientres                1 [11]
# PesoCerdos                  1 [12]
# PesoNetoCanal               1 [13]
# Nacimiento                  1 [14]
# CodigoMatadero              1 [15]
# NroAnimales                 1 [16]
# PesoEntradaEntrada          0
# TipoPesada                  1 [18]
# PesoSalida                122
# DescripcionArticulo         8
# FechaEntrada                1 [21]
# FechaSalida                62
# TipoPesadaSalida            1
# EstadoProductoSalida        2
# dtype: int64

In [30]:
#Función checkeo valores repetidos para valores por partida

def checkRepValPerPart(df):

    checkValues = [1,3,4,6,8,9,10,11,12,13,14,15,16,18,21]

    for partidas in df["Partida"].unique():
        for index in checkValues:
            if df[df["Partida"]==partidas].nunique()[index]==1 or df[df["Partida"]==partidas].nunique()[index]==0:
                continue
            else:
                print("Partidas "+ str(partidas) +" tiene repetidas en index "+ str(index))

In [31]:


checkRepValPerPart(df_InDt3)


Partidas 1584 tiene repetidas en index 1
Partidas 1584 tiene repetidas en index 4
Partidas 1584 tiene repetidas en index 10
Partidas 1584 tiene repetidas en index 11
Partidas 1584 tiene repetidas en index 12
Partidas 1584 tiene repetidas en index 13
Partidas 1584 tiene repetidas en index 14
Partidas 1584 tiene repetidas en index 16
Partidas 1584 tiene repetidas en index 21
Partidas 223 tiene repetidas en index 1
Partidas 223 tiene repetidas en index 3
Partidas 223 tiene repetidas en index 10
Partidas 223 tiene repetidas en index 11
Partidas 223 tiene repetidas en index 12
Partidas 223 tiene repetidas en index 13
Partidas 223 tiene repetidas en index 14
Partidas 223 tiene repetidas en index 16
Partidas 223 tiene repetidas en index 21
Partidas 446 tiene repetidas en index 1
Partidas 446 tiene repetidas en index 3
Partidas 446 tiene repetidas en index 10
Partidas 446 tiene repetidas en index 11
Partidas 446 tiene repetidas en index 12
Partidas 446 tiene repetidas en index 13
Partidas 446 

Partidas 2020 tiene repetidas en index 1
Partidas 2020 tiene repetidas en index 3
Partidas 2020 tiene repetidas en index 4
Partidas 2020 tiene repetidas en index 10
Partidas 2020 tiene repetidas en index 11
Partidas 2020 tiene repetidas en index 12
Partidas 2020 tiene repetidas en index 13
Partidas 2020 tiene repetidas en index 14
Partidas 2020 tiene repetidas en index 16
Partidas 2020 tiene repetidas en index 21
Partidas 2021 tiene repetidas en index 1
Partidas 2021 tiene repetidas en index 10
Partidas 2021 tiene repetidas en index 11
Partidas 2021 tiene repetidas en index 12
Partidas 2021 tiene repetidas en index 13
Partidas 2021 tiene repetidas en index 14
Partidas 2021 tiene repetidas en index 16
Partidas 2021 tiene repetidas en index 21
Partidas 2022 tiene repetidas en index 1
Partidas 2022 tiene repetidas en index 10
Partidas 2022 tiene repetidas en index 11
Partidas 2022 tiene repetidas en index 12
Partidas 2022 tiene repetidas en index 13
Partidas 2022 tiene repetidas en index 

In [32]:
# Comprobando repeticiones y preguntando al proveedor de los datos llegamos a la conclusion
# De que los números de partida no son únicos y por tanto los renombraremos mediante
# concatenación de año-mes/nºpartida
df_InDt4 = df_InDt3.copy()
df_InDt4['FechaEntrada'] =  pd.to_datetime(df_InDt4['FechaEntrada'])

df_InDt4["Partida"]= df_InDt4['FechaEntrada'].dt.strftime('%Y-%m').astype("str").map(str) + "/" + df_InDt4["Partida"].astype("str")
df_InDt4

,TipoArticuloEscandallo,Añada,Partida,Raza,Alimentacion,NumeroSerieLc,AcidoOleico,AcidoPalmitico,AcidoEstearico,AcidoLinoleico,...,CodigoMatadero,NroAnimales,PesoEntradaEntrada,TipoPesada,PesoSalida,DescripcionArticulo,FechaEntrada,FechaSalida,TipoPesadaSalida,EstadoProductoSalida
480,0,2014,2014-03/1025,0,0,140121025013,57.4,18.99,8.23,8.85,...,0,130,NaN,0,8.755,JAMON DE BELLOTA IBERICO PASSION,2014-03-20,2018-06-04,0,0
481,0,2014,2014-03/1025,0,0,140121025117,57.4,18.99,8.23,8.85,...,0,130,NaN,0,7.755,"JAMON BELLOTA IBERICO 75% RAZA IBERICA75% 7,0-...",2014-03-20,2018-10-31,0,0
482,0,2014,2014-03/1025,0,0,140121025205,57.4,18.99,8.23,8.85,...,0,130,NaN,0,7.755,"JAMON BELLOTA IBERICO 75% RAZA IBERICA75% 7,0-...",2014-03-20,2018-10-31,0,0
483,0,2014,2014-03/1025,0,0,140121025235,57.4,18.99,8.23,8.85,...,0,130,NaN,0,7.400,JAMON IBERICO BELLOTA PIEZA UN DONIBERIIBERICO...,2014-03-20,2017-12-01,0,0
484,1,2014,2014-03/1025,0,0,141121025162,57.4,18.99,8.23,8.85,...,0,130,NaN,0,6.640,PALETA DE BELLOTA IBÉRICA PASSION,2014-03-20,2017-08-28,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158723,1,2018,2018-02/999,1,0,237030576292,NaN,NaN,NaN,NaN,...,0,65,NaN,0,6.383,PALETA DE BELLOTA IBÉRICA,2018-02-12,2021-04-13,0,0
158724,1,2018,2018-02/999,1,0,237030576292,NaN,NaN,NaN,NaN,...,0,65,NaN,0,3.160,PALETA DE BELLOTA IBÉRICA,2018-02-12,2021-04-13,0,0
158725,1,2018,2018-02/999,1,0,237030576292,NaN,NaN,NaN,NaN,...,0,65,NaN,0,6.383,PALETA DE BELLOTA IBÉRICA DESH.,2018-02-12,2021-04-16,1,0
158726,1,2018,2018-02/999,1,0,237030576292,NaN,NaN,NaN,NaN,...,0,65,NaN,0,3.160,PALETA DE BELLOTA IBÉRICA DESH.,2018-02-12,2021-04-16,1,0


In [33]:
#Volvemos a checkear
checkRepValPerPart(df_InDt4)

Partidas 2021-04/212089 tiene repetidas en index 18
Partidas 2017-01/432 tiene repetidas en index 18
Partidas 2017-02/1303 tiene repetidas en index 18


In [34]:
# Creamos el dataframe con las columnas que corresponden a la tabla "partida"
checkValues = [1,2,3,4,6,7,8,9,10,11,12,13,14,15,16,18,21]
colsPartidas = []
for pos in checkValues:
    colsPartidas.append(df_InDt4.columns[pos])

df_Ptda1 = df_InDt4.loc[:,colsPartidas]
df_Ptda1

,Añada,Partida,Raza,Alimentacion,AcidoOleico,AcidoPalmitico,AcidoEstearico,AcidoLinoleico,PesoVarios,Pesovientres,PesoCerdos,PesoNetoCanal,Nacimiento,CodigoMatadero,NroAnimales,TipoPesada,FechaEntrada
480,2014,2014-03/1025,0,0,57.4,18.99,8.23,8.85,10260.824,17648.8,21300.0,17295.824,2012-05-20,0,130,0,2014-03-20
481,2014,2014-03/1025,0,0,57.4,18.99,8.23,8.85,10260.824,17648.8,21300.0,17295.824,2012-05-20,0,130,0,2014-03-20
482,2014,2014-03/1025,0,0,57.4,18.99,8.23,8.85,10260.824,17648.8,21300.0,17295.824,2012-05-20,0,130,0,2014-03-20
483,2014,2014-03/1025,0,0,57.4,18.99,8.23,8.85,10260.824,17648.8,21300.0,17295.824,2012-05-20,0,130,0,2014-03-20
484,2014,2014-03/1025,0,0,57.4,18.99,8.23,8.85,10260.824,17648.8,21300.0,17295.824,2012-05-20,0,130,0,2014-03-20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158723,2018,2018-02/999,1,0,NaN,NaN,NaN,NaN,5244.962,9846.9,11596.0,9649.962,2016-10-05,0,65,0,2018-02-12
158724,2018,2018-02/999,1,0,NaN,NaN,NaN,NaN,5244.962,9846.9,11596.0,9649.962,2016-10-05,0,65,0,2018-02-12
158725,2018,2018-02/999,1,0,NaN,NaN,NaN,NaN,5244.962,9846.9,11596.0,9649.962,2016-10-05,0,65,0,2018-02-12
158726,2018,2018-02/999,1,0,NaN,NaN,NaN,NaN,5244.962,9846.9,11596.0,9649.962,2016-10-05,0,65,0,2018-02-12


In [35]:
df_Ptda1.isna().sum()

Añada                 0
Partida               0
Raza                  0
Alimentacion          0
AcidoOleico       86077
AcidoPalmitico    86077
AcidoEstearico    86077
AcidoLinoleico    86077
PesoVarios         1444
Pesovientres       1444
PesoCerdos         1444
PesoNetoCanal      1444
Nacimiento            0
CodigoMatadero        0
NroAnimales           0
TipoPesada            0
FechaEntrada          0
dtype: int64

In [36]:
# Para crear que funcione correctamente el groupby hay que sustituir los valores nan por un
# valor alternativo que devolveremos posteriormente a valor nan

df_Ptda2 = df_Ptda1.copy()
df_Ptda2["AcidoOleico"] = df_Ptda2["AcidoOleico"].fillna(-1) 
df_Ptda2["AcidoPalmitico"] = df_Ptda2["AcidoPalmitico"].fillna(-1) 
df_Ptda2["AcidoEstearico"] = df_Ptda2["AcidoEstearico"].fillna(-1) 
df_Ptda2["AcidoLinoleico"] = df_Ptda2["AcidoLinoleico"].fillna(-1) 
df_Ptda2["PesoVarios"] = df_Ptda2["PesoVarios"].fillna(-1) 
df_Ptda2["Pesovientres"] = df_Ptda2["Pesovientres"].fillna(-1)
df_Ptda2["PesoCerdos"] = df_Ptda2["PesoCerdos"].fillna(-1)  
df_Ptda2["PesoNetoCanal"] = df_Ptda2["PesoNetoCanal"].fillna(-1)


df_Ptda2

,Añada,Partida,Raza,Alimentacion,AcidoOleico,AcidoPalmitico,AcidoEstearico,AcidoLinoleico,PesoVarios,Pesovientres,PesoCerdos,PesoNetoCanal,Nacimiento,CodigoMatadero,NroAnimales,TipoPesada,FechaEntrada
480,2014,2014-03/1025,0,0,57.4,18.99,8.23,8.85,10260.824,17648.8,21300.0,17295.824,2012-05-20,0,130,0,2014-03-20
481,2014,2014-03/1025,0,0,57.4,18.99,8.23,8.85,10260.824,17648.8,21300.0,17295.824,2012-05-20,0,130,0,2014-03-20
482,2014,2014-03/1025,0,0,57.4,18.99,8.23,8.85,10260.824,17648.8,21300.0,17295.824,2012-05-20,0,130,0,2014-03-20
483,2014,2014-03/1025,0,0,57.4,18.99,8.23,8.85,10260.824,17648.8,21300.0,17295.824,2012-05-20,0,130,0,2014-03-20
484,2014,2014-03/1025,0,0,57.4,18.99,8.23,8.85,10260.824,17648.8,21300.0,17295.824,2012-05-20,0,130,0,2014-03-20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158723,2018,2018-02/999,1,0,-1.0,-1.00,-1.00,-1.00,5244.962,9846.9,11596.0,9649.962,2016-10-05,0,65,0,2018-02-12
158724,2018,2018-02/999,1,0,-1.0,-1.00,-1.00,-1.00,5244.962,9846.9,11596.0,9649.962,2016-10-05,0,65,0,2018-02-12
158725,2018,2018-02/999,1,0,-1.0,-1.00,-1.00,-1.00,5244.962,9846.9,11596.0,9649.962,2016-10-05,0,65,0,2018-02-12
158726,2018,2018-02/999,1,0,-1.0,-1.00,-1.00,-1.00,5244.962,9846.9,11596.0,9649.962,2016-10-05,0,65,0,2018-02-12


In [37]:
df_Ptda3 = df_Ptda2.copy()
df_Ptda3= df_Ptda3.groupby("Partida").agg(lambda x: x.mode())


df_Ptda3 = df_Ptda3.reset_index()
df_Ptda3.insert(0,"IdPartida",df_Ptda3.index,True)

df_Ptda3

,IdPartida,Partida,Añada,Raza,Alimentacion,AcidoOleico,AcidoPalmitico,AcidoEstearico,AcidoLinoleico,PesoVarios,Pesovientres,PesoCerdos,PesoNetoCanal,Nacimiento,CodigoMatadero,NroAnimales,TipoPesada,FechaEntrada
0,0,2013-01/124,2013,0,0,55.50,20.76,8.19,8.40,7654.786,13145.7,16180.00,12882.786,2011-05-20,0,100,0,2013-01-14
1,1,2013-01/165,2013,0,2,-1.00,-1.00,-1.00,-1.00,7669.250,15212.5,18400.00,14908.250,2012-10-01,0,120,0,2013-01-16
2,2,2013-01/223,2013,0,0,53.66,20.60,10.75,9.16,4886.350,8657.5,10800.00,8484.350,2011-09-15,0,65,0,2013-01-21
3,3,2013-01/298,2013,0,2,-1.00,-1.00,-1.00,-1.00,8519.144,17182.8,20700.00,16839.144,2012-10-01,0,145,0,2013-01-24
4,4,2013-01/3,2013,0,0,54.60,20.70,10.10,8.30,5233.422,9673.9,12160.00,9480.422,2011-04-30,0,70,0,2013-01-22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
407,407,2021-02/211291,2021,2,0,56.55,19.55,7.95,9.34,2192.852,3897.4,4744.40,3819.452,2019-09-25,0,30,1,2021-02-22
408,408,2021-02/211378,2021,0,1,-1.00,-1.00,-1.00,-1.00,7960.544,15572.8,18761.43,15261.344,2019-05-29,0,115,1,2021-02-26
409,409,2021-03/211852,2021,0,1,-1.00,-1.00,-1.00,-1.00,9828.216,18814.2,22860.00,18437.916,2019-11-20,0,140,1,2021-03-25
410,410,2021-03/211921,2021,0,1,-1.00,-1.00,-1.00,-1.00,8442.200,16980.0,20700.00,16640.400,2020-01-16,0,130,1,2021-03-29


In [38]:
df_InDt5 = df_InDt4.copy()
IdPart = zip(df_Ptda3["Partida"],df_Ptda3["IdPartida"])
PartID = dict(IdPart)
PartID

df_InDt5["Partida"].replace(to_replace=PartID, inplace=True)
df_InDt5["Partida"].unique()

array([ 52,  53, 157, 161, 296,   0, 163, 175, 176, 179,  75,  76,  77,
        78, 374, 375, 376, 377, 378, 379, 380, 381, 382, 383,  32,  33,
       394, 401, 402, 406,   2,  79,  80,  34,  35,   4,  81,  82,  36,
        83,  84,  85,  86,  87,   5,   6,   7,   8,  37,  38,  89,  10,
        39,  40,  41,  42,  90,  13,  91,  14,  15,  16,  17,  92,  93,
        44,  18,  45,  46,  96,  98,  94,  95,  19, 164, 165, 166,  47,
        48,  99,  50, 100, 101,  20,  21, 169,  51, 102,  54,  55,  56,
       103, 104, 106,  57, 108, 109,  58,  59, 110, 173,  60, 111, 391,
       392, 393, 397, 400, 410, 408, 409, 411, 398, 389, 390, 395, 396,
       399, 403, 404, 405, 407, 210, 228, 133, 290, 291, 222, 292, 223,
        22,  23,  24, 334, 155, 156,  25, 224,  26, 335, 336,  61, 112,
       158, 159, 160, 225, 293, 294, 226, 295,  27,  28,  29, 337, 162,
       338, 339, 113, 227, 229, 297, 298, 200, 201, 202, 340, 341, 114,
        62,  63,  64, 230, 299, 231, 232, 233, 115, 116, 117, 23

In [39]:
df_Ptda3["Nacimiento"] = df_Ptda3["Nacimiento"].astype("str")
df_Ptda3["FechaEntrada"] = df_Ptda3["FechaEntrada"].astype("str")
df_Ptda3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 412 entries, 0 to 411
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   IdPartida       412 non-null    int64  
 1   Partida         412 non-null    object 
 2   Añada           412 non-null    int64  
 3   Raza            412 non-null    int64  
 4   Alimentacion    412 non-null    int64  
 5   AcidoOleico     412 non-null    float64
 6   AcidoPalmitico  412 non-null    float64
 7   AcidoEstearico  412 non-null    float64
 8   AcidoLinoleico  412 non-null    float64
 9   PesoVarios      412 non-null    float64
 10  Pesovientres    412 non-null    float64
 11  PesoCerdos      412 non-null    float64
 12  PesoNetoCanal   412 non-null    float64
 13  Nacimiento      412 non-null    object 
 14  CodigoMatadero  412 non-null    int64  
 15  NroAnimales     412 non-null    int64  
 16  TipoPesada      412 non-null    int64  
 17  FechaEntrada    412 non-null    obj

In [ ]:
# def IntroTablaPartida(dfColumna, nombreColumna):
#     import mysql.connector
    
#     nombreColumna

#     count = 0
#     for values in dfColumna:
#         sqlcategoria = "INSERT INTO factoria_jamones.partida("+str(nombreColumna)+") VALUES (%s)"
#         conex = mysql.connector.connect(
#                 host = "localhost",
#                 user = "root", 
#                 password = "",
#                 database ="factoria_jamones",
#                 port = 3306,
#                 autocommit=True)
#         transaccion = conex.cursor()
#         try:
#             transaccion.execute(sqlcategoria, values)
#         except: 
#             print(str(nombreColumna)+ " número "+ str(count+1) +" ya existe en base de datos")
#             count+=1

In [ ]:
# def IntroTablaPartida(df):
#     import mysql.connector

#     tuples = [tuple(x) for x in df.values]
        
#     sqlcategoria = "INSERT INTO factoria_jamones.partida VALUES ("+(len(df.columns)-1)*"%s,"+"%s)"
#     conex = mysql.connector.connect(
#             host = "localhost",
#             user = "root", 
#             password = "",
#             database ="factoria_jamones",
#             port = 3306,
#             autocommit=True)
#     transaccion = conex.cursor()
#     transaccion.executemany(sqlcategoria, tuples)

In [ ]:
def IntroTablaPartida(df):
    import mysql.connector
        
    sqlcategoria = "INSERT INTO factoria_jamones.partida VALUES ("+(len(df.columns)-1)*"%s,"+"%s)"
    conex = mysql.connector.connect(
            host = "localhost",
            user = "root", 
            password = "",
            database ="factoria_jamones",
            port = 3306,
            autocommit=True)
    transaccion = conex.cursor()
    transaccion.executemany(sqlcategoria, df.values.tolist())
    conex.close()


In [ ]:
IntroTablaPartida(df_Ptda3)

In [40]:


# Creamos el dataframe con las columnas que corresponden a la tabla "partida"
checkValues = [0,2,5,17,19,22,23,24]
colsPartidas = []
for pos in checkValues:
    colsPartidas.append(df_InDt4.columns[pos])

df_Pza1 = df_InDt5.loc[:,colsPartidas]

df_Pza1.insert(0,"IdPieza",df_Pza1.index,True)
df_Pza1["FechaSalida"] = df_Pza1["FechaSalida"].astype("str")
df_Pza1["PesoEntradaEntrada"] = df_Pza1["PesoEntradaEntrada"].fillna(-1)
df_Pza1["PesoSalida"] = df_Pza1["PesoSalida"].fillna(-1)

df_Pza1["IdPieza"].unique()
# len(df_Pza1["IdPieza"])

array([   480,    481,    482, ..., 158725, 158726, 158727], dtype=int64)

In [ ]:
conex = mysql.connector.connect(
        host = "localhost",
        user = "root", 
        password = "",
        database ="factoria_jamones",
        port = 3306,
        autocommit=True)
conex.close()

In [ ]:
def IntroTablaPieza(df):
    import mysql.connector
        
    sqlcategoria = "INSERT INTO factoria_jamones.pieza VALUES ("+(len(df.columns)-1)*"%s,"+"%s)"
    conex = mysql.connector.connect(
            host = "localhost",
            user = "root", 
            password = "",
            database ="factoria_jamones",
            port = 3306,
            autocommit=True)
    transaccion = conex.cursor()
    transaccion.executemany(sqlcategoria, df.values.tolist())
    conex.close()

In [45]:
def IntroTablaPieza(df, IdColumn):
    import mysql.connector
    
    sqlcategoria = "INSERT INTO factoria_jamones.pieza VALUES ("+(len(df.columns)-1)*"%s,"+"%s)"
    conex = mysql.connector.connect(
            host = "localhost",
            user = "root", 
            password = "",
            database ="factoria_jamones",
            port = 3306,
            autocommit=True)
    
    
    dicc = df.set_index(IdColumn).T.to_dict('list')
    count = 0
    
    for keys in dicc:        
        transaccion = conex.cursor()
        try:
            transaccion.execute(sqlcategoria, (keys, dicc[keys][0],dicc[keys][1],dicc[keys][2],dicc[keys][3],dicc[keys][4],dicc[keys][5],dicc[keys][6],dicc[keys][7]))
        except Exception as e: 
#             print(str("registro número "+ str(count+1) +" ya existe en base de datos"))
            print(e)
        count+=1
    conex.close()

In [49]:
df_Pza1.shape

(120481, 9)

In [46]:
IntroTablaPieza(df_Pza1,"IdPieza")

1062 (23000): Duplicate entry '480' for key 'PRIMARY'
1062 (23000): Duplicate entry '481' for key 'PRIMARY'
1062 (23000): Duplicate entry '482' for key 'PRIMARY'
1062 (23000): Duplicate entry '483' for key 'PRIMARY'
1062 (23000): Duplicate entry '484' for key 'PRIMARY'
1062 (23000): Duplicate entry '485' for key 'PRIMARY'
1062 (23000): Duplicate entry '486' for key 'PRIMARY'
1062 (23000): Duplicate entry '487' for key 'PRIMARY'
1062 (23000): Duplicate entry '488' for key 'PRIMARY'
1062 (23000): Duplicate entry '489' for key 'PRIMARY'
1062 (23000): Duplicate entry '490' for key 'PRIMARY'
1062 (23000): Duplicate entry '491' for key 'PRIMARY'
1062 (23000): Duplicate entry '492' for key 'PRIMARY'
1062 (23000): Duplicate entry '493' for key 'PRIMARY'
1062 (23000): Duplicate entry '494' for key 'PRIMARY'
1062 (23000): Duplicate entry '495' for key 'PRIMARY'
1062 (23000): Duplicate entry '496' for key 'PRIMARY'
1062 (23000): Duplicate entry '497' for key 'PRIMARY'
1062 (23000): Duplicate entr

1062 (23000): Duplicate entry '871' for key 'PRIMARY'
1062 (23000): Duplicate entry '872' for key 'PRIMARY'
1062 (23000): Duplicate entry '873' for key 'PRIMARY'
1062 (23000): Duplicate entry '874' for key 'PRIMARY'
1062 (23000): Duplicate entry '875' for key 'PRIMARY'
1062 (23000): Duplicate entry '876' for key 'PRIMARY'
1062 (23000): Duplicate entry '877' for key 'PRIMARY'
1062 (23000): Duplicate entry '878' for key 'PRIMARY'
1062 (23000): Duplicate entry '879' for key 'PRIMARY'
1062 (23000): Duplicate entry '880' for key 'PRIMARY'
1062 (23000): Duplicate entry '881' for key 'PRIMARY'
1062 (23000): Duplicate entry '882' for key 'PRIMARY'
1062 (23000): Duplicate entry '883' for key 'PRIMARY'
1062 (23000): Duplicate entry '884' for key 'PRIMARY'
1062 (23000): Duplicate entry '885' for key 'PRIMARY'
1062 (23000): Duplicate entry '886' for key 'PRIMARY'
1062 (23000): Duplicate entry '887' for key 'PRIMARY'
1062 (23000): Duplicate entry '888' for key 'PRIMARY'
1062 (23000): Duplicate entr

1062 (23000): Duplicate entry '1176' for key 'PRIMARY'
1062 (23000): Duplicate entry '1177' for key 'PRIMARY'
1062 (23000): Duplicate entry '1178' for key 'PRIMARY'
1062 (23000): Duplicate entry '1179' for key 'PRIMARY'
1062 (23000): Duplicate entry '1180' for key 'PRIMARY'
1062 (23000): Duplicate entry '1181' for key 'PRIMARY'
1062 (23000): Duplicate entry '1182' for key 'PRIMARY'
1062 (23000): Duplicate entry '1183' for key 'PRIMARY'
1062 (23000): Duplicate entry '1184' for key 'PRIMARY'
1062 (23000): Duplicate entry '1185' for key 'PRIMARY'
1062 (23000): Duplicate entry '1186' for key 'PRIMARY'
1062 (23000): Duplicate entry '1187' for key 'PRIMARY'
1062 (23000): Duplicate entry '1188' for key 'PRIMARY'
1062 (23000): Duplicate entry '1189' for key 'PRIMARY'
1062 (23000): Duplicate entry '1190' for key 'PRIMARY'
1062 (23000): Duplicate entry '1191' for key 'PRIMARY'
1062 (23000): Duplicate entry '1192' for key 'PRIMARY'
1062 (23000): Duplicate entry '1193' for key 'PRIMARY'
1062 (2300

1062 (23000): Duplicate entry '2444' for key 'PRIMARY'
1062 (23000): Duplicate entry '2445' for key 'PRIMARY'
1062 (23000): Duplicate entry '2446' for key 'PRIMARY'
1062 (23000): Duplicate entry '2447' for key 'PRIMARY'
1062 (23000): Duplicate entry '2448' for key 'PRIMARY'
1062 (23000): Duplicate entry '2449' for key 'PRIMARY'
1062 (23000): Duplicate entry '2450' for key 'PRIMARY'
1062 (23000): Duplicate entry '2451' for key 'PRIMARY'
1062 (23000): Duplicate entry '2452' for key 'PRIMARY'
1062 (23000): Duplicate entry '2453' for key 'PRIMARY'
1062 (23000): Duplicate entry '2454' for key 'PRIMARY'
1062 (23000): Duplicate entry '2455' for key 'PRIMARY'
1062 (23000): Duplicate entry '2456' for key 'PRIMARY'
1062 (23000): Duplicate entry '2457' for key 'PRIMARY'
1062 (23000): Duplicate entry '2458' for key 'PRIMARY'
1062 (23000): Duplicate entry '2459' for key 'PRIMARY'
1062 (23000): Duplicate entry '2460' for key 'PRIMARY'
1062 (23000): Duplicate entry '2461' for key 'PRIMARY'
1062 (2300

1062 (23000): Duplicate entry '2828' for key 'PRIMARY'
1062 (23000): Duplicate entry '2829' for key 'PRIMARY'
1062 (23000): Duplicate entry '2830' for key 'PRIMARY'
1062 (23000): Duplicate entry '2831' for key 'PRIMARY'
1062 (23000): Duplicate entry '2832' for key 'PRIMARY'
1062 (23000): Duplicate entry '2833' for key 'PRIMARY'
1062 (23000): Duplicate entry '2834' for key 'PRIMARY'
1062 (23000): Duplicate entry '2835' for key 'PRIMARY'
1062 (23000): Duplicate entry '2836' for key 'PRIMARY'
1062 (23000): Duplicate entry '2837' for key 'PRIMARY'
1062 (23000): Duplicate entry '2838' for key 'PRIMARY'
1062 (23000): Duplicate entry '2839' for key 'PRIMARY'
1062 (23000): Duplicate entry '2840' for key 'PRIMARY'
1062 (23000): Duplicate entry '2841' for key 'PRIMARY'
1062 (23000): Duplicate entry '2842' for key 'PRIMARY'
1062 (23000): Duplicate entry '2843' for key 'PRIMARY'
1062 (23000): Duplicate entry '2844' for key 'PRIMARY'
1062 (23000): Duplicate entry '2845' for key 'PRIMARY'
1062 (2300

...

1062 (23000): Duplicate entry '29547' for key 'PRIMARY'
1062 (23000): Duplicate entry '29548' for key 'PRIMARY'
1062 (23000): Duplicate entry '29549' for key 'PRIMARY'
1062 (23000): Duplicate entry '29550' for key 'PRIMARY'
1062 (23000): Duplicate entry '29551' for key 'PRIMARY'
1062 (23000): Duplicate entry '29552' for key 'PRIMARY'
1062 (23000): Duplicate entry '29553' for key 'PRIMARY'
1062 (23000): Duplicate entry '29554' for key 'PRIMARY'
1062 (23000): Duplicate entry '29555' for key 'PRIMARY'
1062 (23000): Duplicate entry '29556' for key 'PRIMARY'
1062 (23000): Duplicate entry '29557' for key 'PRIMARY'
1062 (23000): Duplicate entry '29558' for key 'PRIMARY'
1062 (23000): Duplicate entry '29559' for key 'PRIMARY'
1062 (23000): Duplicate entry '29560' for key 'PRIMARY'
1062 (23000): Duplicate entry '29561' for key 'PRIMARY'
1062 (23000): Duplicate entry '29562' for key 'PRIMARY'
1062 (23000): Duplicate entry '29563' for key 'PRIMARY'
1062 (23000): Duplicate entry '29564' for key 'P

1062 (23000): Duplicate entry '29891' for key 'PRIMARY'
1062 (23000): Duplicate entry '29892' for key 'PRIMARY'
1062 (23000): Duplicate entry '29893' for key 'PRIMARY'
1062 (23000): Duplicate entry '29894' for key 'PRIMARY'
1062 (23000): Duplicate entry '29896' for key 'PRIMARY'
1062 (23000): Duplicate entry '29897' for key 'PRIMARY'
1062 (23000): Duplicate entry '29898' for key 'PRIMARY'
1062 (23000): Duplicate entry '29899' for key 'PRIMARY'
1062 (23000): Duplicate entry '29900' for key 'PRIMARY'
1062 (23000): Duplicate entry '29901' for key 'PRIMARY'
1062 (23000): Duplicate entry '29902' for key 'PRIMARY'
1062 (23000): Duplicate entry '29903' for key 'PRIMARY'
1062 (23000): Duplicate entry '29904' for key 'PRIMARY'
1062 (23000): Duplicate entry '29905' for key 'PRIMARY'
1062 (23000): Duplicate entry '29906' for key 'PRIMARY'
1062 (23000): Duplicate entry '29907' for key 'PRIMARY'
1062 (23000): Duplicate entry '29908' for key 'PRIMARY'
1062 (23000): Duplicate entry '29909' for key 'P

1062 (23000): Duplicate entry '30167' for key 'PRIMARY'
1062 (23000): Duplicate entry '30168' for key 'PRIMARY'
1062 (23000): Duplicate entry '30169' for key 'PRIMARY'
1062 (23000): Duplicate entry '30171' for key 'PRIMARY'
1062 (23000): Duplicate entry '30172' for key 'PRIMARY'
1062 (23000): Duplicate entry '30173' for key 'PRIMARY'
1062 (23000): Duplicate entry '30174' for key 'PRIMARY'
1062 (23000): Duplicate entry '30175' for key 'PRIMARY'
1062 (23000): Duplicate entry '30176' for key 'PRIMARY'
1062 (23000): Duplicate entry '30177' for key 'PRIMARY'
1062 (23000): Duplicate entry '30178' for key 'PRIMARY'
1062 (23000): Duplicate entry '30179' for key 'PRIMARY'
1062 (23000): Duplicate entry '30180' for key 'PRIMARY'
1062 (23000): Duplicate entry '30181' for key 'PRIMARY'
1062 (23000): Duplicate entry '30182' for key 'PRIMARY'
1062 (23000): Duplicate entry '30183' for key 'PRIMARY'
1062 (23000): Duplicate entry '30184' for key 'PRIMARY'
1062 (23000): Duplicate entry '30185' for key 'P

1062 (23000): Duplicate entry '30439' for key 'PRIMARY'
1062 (23000): Duplicate entry '30440' for key 'PRIMARY'
1062 (23000): Duplicate entry '30441' for key 'PRIMARY'
1062 (23000): Duplicate entry '30442' for key 'PRIMARY'
1062 (23000): Duplicate entry '30443' for key 'PRIMARY'
1062 (23000): Duplicate entry '30444' for key 'PRIMARY'
1062 (23000): Duplicate entry '30445' for key 'PRIMARY'
1062 (23000): Duplicate entry '30446' for key 'PRIMARY'
1062 (23000): Duplicate entry '30447' for key 'PRIMARY'
1062 (23000): Duplicate entry '30448' for key 'PRIMARY'
1062 (23000): Duplicate entry '30449' for key 'PRIMARY'
1062 (23000): Duplicate entry '30450' for key 'PRIMARY'
1062 (23000): Duplicate entry '30451' for key 'PRIMARY'
1062 (23000): Duplicate entry '30452' for key 'PRIMARY'
1062 (23000): Duplicate entry '30453' for key 'PRIMARY'
1062 (23000): Duplicate entry '30454' for key 'PRIMARY'
1062 (23000): Duplicate entry '30455' for key 'PRIMARY'
1062 (23000): Duplicate entry '30456' for key 'P

1062 (23000): Duplicate entry '30724' for key 'PRIMARY'
1062 (23000): Duplicate entry '30725' for key 'PRIMARY'
1062 (23000): Duplicate entry '30726' for key 'PRIMARY'
1062 (23000): Duplicate entry '30727' for key 'PRIMARY'
1062 (23000): Duplicate entry '30728' for key 'PRIMARY'
1062 (23000): Duplicate entry '30729' for key 'PRIMARY'
1062 (23000): Duplicate entry '30730' for key 'PRIMARY'
1062 (23000): Duplicate entry '30731' for key 'PRIMARY'
1062 (23000): Duplicate entry '30733' for key 'PRIMARY'
1062 (23000): Duplicate entry '30734' for key 'PRIMARY'
1062 (23000): Duplicate entry '30735' for key 'PRIMARY'
1062 (23000): Duplicate entry '30736' for key 'PRIMARY'
1062 (23000): Duplicate entry '30737' for key 'PRIMARY'
1062 (23000): Duplicate entry '30738' for key 'PRIMARY'
1062 (23000): Duplicate entry '30739' for key 'PRIMARY'
1062 (23000): Duplicate entry '30740' for key 'PRIMARY'
1062 (23000): Duplicate entry '30741' for key 'PRIMARY'
1062 (23000): Duplicate entry '30742' for key 'P

In [ ]:
def guardar_juego(juegos):
    
    import mysql.connector
    count = 0

        

            
# CREATE TABLE `matadero` (
#   `id_matadero` integer,
#   PRIMARY KEY (`id_matadero`)
# );
    
    #INTRODUCIR DATOS TABLA MATADERO   
    for mataderos in df_v_final[matadero].unique:
        sqlmatadero = "INSERT INTO factoria_jamones.matadero (id_matadero) VALUES (%s)"
        conex = mysql.connector.connect(
                host = "localhost",
                user = "root", 
                password = "",
                database ="factoria_jamones",
                port = 3306,
                autocommit=True)
        transaccion = conex.cursor()
        try:
            transaccion.execute(sqlmatadero, (mataderos))
        except: 
            print("raza número "+ str(count+1) +" ya existe en base de datos")
            count+=1
            
# CREATE TABLE `matadero` (
#   `id_matadero` integer,
#   PRIMARY KEY (`id_matadero`)
# );
    
    #INTRODUCIR DATOS TABLA MATADERO   
    for mataderos in df_v_final[matadero].unique:
        sqlmatadero = "INSERT INTO factoria_jamones.matadero (id_matadero) VALUES (%s)"
        conex = mysql.connector.connect(
                host = "localhost",
                user = "root", 
                password = "",
                database ="factoria_jamones",
                port = 3306,
                autocommit=True)
        transaccion = conex.cursor()
        try:
            transaccion.execute(sqlmatadero, (mataderos))
        except: 
            print("raza número "+ str(count+1) +" ya existe en base de datos")
            count+=1
            
        #INTRODUCIR DATOS TABLA CATEGORIAS
        for categoria in juegos[key]["recop_categorias"]:
            sqlcategorias = "INSERT INTO boardgameanalisis.categorias VALUES (%s,%s)"
            try:
                transaccion.execute(sqlcategorias,
                                    (categoria,
                                     juegos[key]["recop_categorias"][categoria]))
            except: 
                print("Categoria ya existe en BBDD")

            
        #INTRODUCIR DATOS TABLA MECANICAS
        for mecanica in juegos[key]["recop_mecanicas"]:
            sqlmecanicas = "INSERT INTO boardgameanalisis.mecanicas VALUES (%s,%s)"
            try:
                transaccion.execute(sqlmecanicas, 
                                    (mecanica,
                                    juegos[key]["recop_mecanicas"][mecanica]))
            except: 
                print("Mecanica ya existe en BBDD")

            
        #INTRODUCIR DATOS TABLA CATEGORIAS_JUEGO
        for cats in juegos[key]["recop_categorias"]:
            sqlcat_juego = "INSERT INTO boardgameanalisis.categoria_juego VALUES (%s,%s)"
            try:
                transaccion.execute(sqlcat_juego, 
                                    (int(key),int(cats)))
            except: 
                print("Relacion Juego-Categoria ya existe en BBDD")

            
            
            
        #INTRODUCIR DATOS TABLA MECANICAS_JUEGO
        for mecs in juegos[key]["recop_mecanicas"]:
            sqlmec_juego = "INSERT INTO boardgameanalisis.mecanicas_juego VALUES (%s,%s)"
            try:
                transaccion.execute(sqlmec_juego, 
                                    (int(key),int(mecs)))
            except: 
                print("Relacion Juego-Mecanica ya existe en BBDD")

In [ ]:
df_v1

In [ ]:
# PASO 2 ELIMINAR FECHAS DOBLES Y VACÍAS EN df_v1["Nacimiento"] 

df_v1 = df_v1.drop(df_v1[df_v1['Nacimiento'].str.len()>10].index)
df_v1 = df_v1.drop(df_v1[df_v1['Nacimiento'].isna()].index)
df_v1.info()


In [ ]:
# PASO 3 Convertir columnas "Nacimiento", "FechaEntrada" y "FechaSalida" en formato fecha

df_v1['FechaEntrada'] =  pd.to_datetime(df_v1['FechaEntrada'])
df_v1['FechaSalida'] =  pd.to_datetime(df_v1['FechaSalida'])
df_v1['Nacimiento'] =  pd.to_datetime(df_v1['Nacimiento'])

In [ ]:
# PASO 4 CREAMOS COLUMNA TiempoMaduracion CON FECHAS DE ENTRADA Y SALIDA DE LA PIEZA

# TiempoMaduracion EN DÍAS
df_v1["TiempoMaduracion"]= (df_v1['FechaSalida']-df_v1['FechaEntrada'])
df_v1["TiempoMaduracion"]

# TiempoMaduracionMESES EN MESES
from datetime import datetime as dt

df_v1["TiempoMaduracionMESES"]=round((df_v1["TiempoMaduracion"].dt.days)/30.41).astype(int)
df_v1["TiempoMaduracionMESES"]

In [ ]:
# PASO 5 CREAMOS COLUMNA EdadSacrificio CON FECHAS DE NACIMIENTO Y ENTRADA DE LA PIEZA

# EdadSacrificio EN DÍAS
df_v1['EdadSacrificio']= df_v1['FechaEntrada'] - df_v1['Nacimiento']
df_v1['EdadSacrificio']

# EdadSacrificioMESES EN MESES
df_v1["EdadSacrificioMESES"]=round((df_v1["EdadSacrificio"].dt.days)/30.41).astype(int)
df_v1["EdadSacrificioMESES"]